In [1]:
import           Control.Monad.Trans.State.Strict
import qualified Data.IntMap.Strict  as M
import qualified Data.Vector         as V

import           Control.Monad (forM_, when)
import           Data.Maybe (isNothing, fromJust)
import           Prelude hiding (lookup)

In [2]:
graph = M.fromList
    [ (1, [2])
    , (2, [3])
    , (3, [1])
    , (4, [2, 3, 5])
    , (5, [4, 6])
    , (6, [7, 3])
    , (7, [6])
    , (8, [5, 7, 8])
    ]

In [3]:
data TState = TState
  { index    :: Int
  , stack    :: [Int]
  , stackSet :: V.Vector Bool
  , indices  :: V.Vector (Maybe Int)
  , lowlinks :: V.Vector (Maybe Int)
  , output   :: [[Int]]
  } deriving (Eq, Show)

In [4]:
tarjan :: M.IntMap [Int] -> [[Int]] 
tarjan graph = output $ flip execState (TState 0 [] newVector' newVector newVector []) $ do
    forM_ (M.keys graph) $ \v -> do
        is <- gets indices
        when (isNothing (is V.! v)) $ strongConnect v graph
     where 
        size = (fst $ M.findMax graph) + 1
        newVector  = V.replicate size Nothing :: V.Vector (Maybe Int)
        newVector' = V.replicate size False

insert :: V.Vector a -> Int -> a -> V.Vector a
insert vector index value = vector V.// [(index, value)]

strongConnect :: Int -> M.IntMap [Int] -> State TState ()
strongConnect v graph = do
    modify' $ \s -> let i  = index s in s
        { index    = i + 1
        , indices  = insert (indices  s) v (Just i)
        , lowlinks = insert (lowlinks s) v (Just i)
        , stack    = v:(stack s)
        , stackSet = insert (stackSet s) v True
        }
    forM_ (graph M.! v) $ \w -> do
        is <- gets indices
        case is V.! w of
            Nothing -> do
                strongConnect w graph
                ls <- gets lowlinks
                let vLowlink = fromJust (ls V.! v)
                let wLowlink = fromJust (ls V.! w)
                modify' $ \s ->
                    s { lowlinks = insert (lowlinks s) v (Just (min vLowlink wLowlink)) }
            Just wIndex -> do
                sSet <- gets stackSet
                when (sSet V.! w) $ do
                    ls <- gets lowlinks
                    let vLowlink = fromJust (ls V.! v)
                    modify' $ \s ->
                        s { lowlinks = insert (lowlinks s) v (Just (min vLowlink wIndex)) }
    s <- get
    let vLowlink = fromJust (lowlinks s V.! v)
    let vIndex   = fromJust (indices s V.! v)
    when (vLowlink == vIndex) $ do
        scc <- addSCC v []
        modify' $ \s -> s
            { output = output s ++ [scc] }

addSCC :: Int -> [Int] -> State TState [Int]
addSCC v scc = do
    w <- pop
    let scc' = w:scc
    if w == v
        then return scc'
        else addSCC v scc'
    where
        pop = do
            s <- get
            let e = head (stack s)
            let stack' = tail $ stack s
            let stackSet' = insert (stackSet s) e False
            put s { stack = stack', stackSet = stackSet' }
            return e

In [5]:
tarjan graph

[[1,2,3],[6,7],[4,5],[8]]